In [13]:
import os
import boto3
import pandas as pd

In [2]:
_AWS_S3_KEY= os.getenv('AWS_S3_KEY')
_AWS_S3_SECRET= os.getenv('AWS_S3_SECRET')

In [3]:
_AWS_S3_SECRET

'2mhUrECgcIuUYo4ZM9/f1Vdlm8wdaI6Fp8e9IYWY'

In [4]:
session= boto3.Session(
    aws_access_key_id=_AWS_S3_KEY,
    aws_secret_access_key=_AWS_S3_SECRET,
)

In [7]:
s3 = session.resource("s3")

In [8]:
s3_bucket= s3.Bucket("anyoneai-datasets")

In [9]:
l = list(s3_bucket.objects.filter(Prefix="credit-data-2010/"))

In [10]:
objs = [item for item in l if item.key[-1] != '/']

In [11]:
for object_summary in s3_bucket.objects.filter(Prefix="credit-data-2010/"):
    try:
        with open(os.path.join(
                '../data', os.path.split(object_summary.key)[-1]
                  ), 'wb') as data:
            s3_bucket.download_fileobj(object_summary.key, data)
    except IsADirectoryError:
        continue

In [122]:
data= pd.read_csv('../data/PAKDD2010_Modeling_Data.txt', encoding= "ISO-8859-1", header= None, delimiter='\t')

/var/folders/7m/ryt4krsd3q58y7d6y0f2_k1w0000gp/T/ipykernel_2170/2582088791.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('../data/PAKDD2010_Modeling_Data.txt', encoding= "ISO-8859-1", header= None, delimiter='\t')


There are two "EDUCATION_LEVEL"

In [123]:
df_variables = pd.read_excel(
    '../data/PAKDD2010_VariablesList.XLS', 
    header= 0, 
    index_col= 0
)

df_variables= df_variables\
    .set_index(fido.index - 1)

In [124]:
df_variables[df_variables.Var_Title == "EDUCATION_LEVEL"].Var_Description

Var_Id
9      Edducational level in gradual order not informed
43    Mate's educational level in gradual order not ...
Name: Var_Description, dtype: object

In [125]:
rename_dict= {column: df_variables.loc[column].Var_Title for column in data.columns}

In [126]:
rename_dict[43] = f"MATE_{rename_dict[43]}" 

In [ ]:
pd.DataFrame.columns

In [141]:
import numpy as np

In [142]:
s_n_unique_values= data.nunique()
s_n_unique_values.name= "# unique values"
s_n_null_values= data.isna().sum()
s_n_null_values.name= "# NULL Values"
s_per_null_values= np.round(data.isna().sum() / data.shape[0] * 100, 2)
s_per_null_values.name= "% NULL Values"

In [145]:
columns_metrics= pd.concat(objs=[s_n_unique_values, s_n_null_values, s_per_null_values], axis= 1)

In [150]:
mask_drop_col_condition_01 = ((columns_metrics["# unique values"] == 1) & (columns_metrics["# NULL Values"] == 0))
mask_drop_col_condition_02 = (columns_metrics["% NULL Values"] > 50)

In [157]:
columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02]

,# unique values,# NULL Values,% NULL Values
CLERK_TYPE,1,0,0.00
QUANT_ADDITIONAL_CARDS,1,0,0.00
EDUCATION_LEVEL,1,0,0.00
FLAG_MOBILE_PHONE,1,0,0.00
PROFESSIONAL_CITY,2236,33783,67.57
PROFESSIONAL_BOROUGH,5057,33783,67.57
MATE_PROFESSION_CODE,18,28884,57.77
MATE_EDUCATION_LEVEL,6,32338,64.68
FLAG_HOME_ADDRESS_DOCUMENT,1,0,0.00
FLAG_RG,1,0,0.00


In [158]:
columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02].index

Index(['CLERK_TYPE', 'QUANT_ADDITIONAL_CARDS', 'EDUCATION_LEVEL',
       'FLAG_MOBILE_PHONE', 'PROFESSIONAL_CITY', 'PROFESSIONAL_BOROUGH',
       'MATE_PROFESSION_CODE', 'MATE_EDUCATION_LEVEL',
       'FLAG_HOME_ADDRESS_DOCUMENT', 'FLAG_RG', 'FLAG_CPF',
       'FLAG_INCOME_PROOF', 'FLAG_ACSP_RECORD'],
      dtype='object')

In [159]:
[
    col for col in data.columns if col not in columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02].index
]

['PAYMENT_DAY',
 'APPLICATION_SUBMISSION_TYPE',
 'POSTAL_ADDRESS_TYPE',
 'SEX',
 'MARITAL_STATUS',
 'QUANT_DEPENDANTS',
 'STATE_OF_BIRTH',
 'CITY_OF_BIRTH',
 'NACIONALITY',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_BOROUGH',
 'FLAG_RESIDENCIAL_PHONE',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'RESIDENCE_TYPE',
 'MONTHS_IN_RESIDENCE',
 'FLAG_EMAIL',
 'PERSONAL_MONTHLY_INCOME',
 'OTHER_INCOMES',
 'FLAG_VISA',
 'FLAG_MASTERCARD',
 'FLAG_DINERS',
 'FLAG_AMERICAN_EXPRESS',
 'FLAG_OTHER_CARDS',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'PERSONAL_ASSETS_VALUE',
 'QUANT_CARS',
 'COMPANY',
 'PROFESSIONAL_STATE',
 'FLAG_PROFESSIONAL_PHONE',
 'PROFESSIONAL_PHONE_AREA_CODE',
 'MONTHS_IN_THE_JOB',
 'PROFESSION_CODE',
 'OCCUPATION_TYPE',
 'PRODUCT',
 'AGE',
 'RESIDENCIAL_ZIP_3',
 'PROFESSIONAL_ZIP_3',
 'TARGET_LABEL_BAD=1']

In [160]:
data[[
    col for col in data.columns if col not in columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02].index
]].corr()[data.columns[-1]]

/var/folders/7m/ryt4krsd3q58y7d6y0f2_k1w0000gp/T/ipykernel_2170/3312967917.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[[


PAYMENT_DAY                       0.070633
POSTAL_ADDRESS_TYPE              -0.002423
MARITAL_STATUS                   -0.030380
QUANT_DEPENDANTS                  0.012157
NACIONALITY                      -0.000952
RESIDENCE_TYPE                    0.019168
MONTHS_IN_RESIDENCE              -0.029336
FLAG_EMAIL                       -0.008067
PERSONAL_MONTHLY_INCOME           0.002819
OTHER_INCOMES                     0.004442
FLAG_VISA                        -0.005398
FLAG_MASTERCARD                  -0.017658
FLAG_DINERS                       0.002240
FLAG_AMERICAN_EXPRESS             0.000337
FLAG_OTHER_CARDS                  0.001410
QUANT_BANKING_ACCOUNTS            0.011997
QUANT_SPECIAL_BANKING_ACCOUNTS    0.011997
PERSONAL_ASSETS_VALUE            -0.003886
QUANT_CARS                        0.011706
MONTHS_IN_THE_JOB                -0.007073
PROFESSION_CODE                  -0.011593
OCCUPATION_TYPE                   0.040216
PRODUCT                           0.019293
AGE        

In [151]:
columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02]

,# unique values,# NULL Values,% NULL Values
CLERK_TYPE,1,0,0.00
QUANT_ADDITIONAL_CARDS,1,0,0.00
EDUCATION_LEVEL,1,0,0.00
FLAG_MOBILE_PHONE,1,0,0.00
PROFESSIONAL_CITY,2236,33783,67.57
PROFESSIONAL_BOROUGH,5057,33783,67.57
MATE_PROFESSION_CODE,18,28884,57.77
MATE_EDUCATION_LEVEL,6,32338,64.68
FLAG_HOME_ADDRESS_DOCUMENT,1,0,0.00
FLAG_RG,1,0,0.00


In [127]:
data = data\
    .rename(columns= rename_dict)\
    .set_index('ID_CLIENT')

### Ed Level

In [102]:
data.MATE_PROFESSION_CODE.unique()

array([nan, 11.,  0.,  9., 16.,  2., 10., 12.,  6., 15., 13.,  7.,  8.,
        5., 17.,  1.,  3.,  4., 14.])

In [92]:
df_variables[df_variables.Var_Title == "PROFESSION_CODE"].Var_Description.values

array(["Applicant's profession code. Encoding not informed"], dtype=object)

In [85]:
data[data.columns[(data.isna().sum() > 0)]].isna().sum()

RESIDENCE_TYPE           1349
MONTHS_IN_RESIDENCE      3777
PROFESSIONAL_CITY       33783
PROFESSIONAL_BOROUGH    33783
PROFESSION_CODE          7756
OCCUPATION_TYPE          7313
MATE_PROFESSION_CODE    28884
MATE_EDUCATION_LEVEL    32338
dtype: int64